# Lighthouse Indicium

# 0.0 Planejamento da Solução (IOT)

## Input (Entrada)

**1. Problema de Negócio**
* Precificar os carros do cliente o mais próximo aos valores do mercado

**2. Conjunto de Dados**
* Base de dados de um marketplace de compra e venda

## Output (Saída)

**1. Predição dos preços dos carros**
* Lista com: id_carro | preço
    
**2. Relatório das variáveis (features)**
* Análise das principais estatísticas descritivas com visualização

**3. Análise Exploratória dos Dados (EDA)**
* 3 hipóteses de negócio
* Perguntas feitas pelo cliente
    - Qual o melhor estado cadastrado na base de dados para se vender um carro de marca popular e por quê?
    - Qual o melhor estado para se comprar uma picape com transmissão automática e por quê?
    - Qual o melhor estado para se comprar carros que ainda estejam dentro da garantia de fábrica e por quê?

## Tasks (Tarefas)

**1. Predição dos preços dos carros**
* Desenvolver modelo a partir da base de dados para predição dos preços dos carros.
    
**2. Relatório das variáveis (features)**
* Análise do comportamento de cada variável (média, mediana, distribuição, etc)

**3. Análise Exploratória dos Dados (EDA)**
* 3 hipóteses de negócio
    - Mapa mental para brainstorm e criação de hipóteses
* Perguntas feitas pelo cliente
    - Qual o melhor estado cadastrado na base de dados para se vender um carro de marca popular e por quê?
        - Quais as principais condições para o estado ser melhor? Preço?
    - Qual o melhor estado para se comprar uma picape com transmissão automática e por quê?
         - Quais as principais condições do veículo/estado para ser melhor? Quilometragem, preço, revisão?
    - Qual o melhor estado para se comprar carros que ainda estejam dentro da garantia de fábrica e por quê?
        - Análise dos carros com garantia de fábrica por estado.

# 0.0 IMPORTS

In [167]:
import pandas as pd
import numpy as np
import seaborn as sns
import xgboost as xgb
import lightgbm as lgbm

from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from IPython.core.display  import HTML

from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics       import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from sklearn.linear_model  import LinearRegression, Lasso
from sklearn.ensemble      import RandomForestRegressor

from feature_engine.encoding import CountFrequencyEncoder

## 0.1 Helper Functions

In [68]:
def descriptive_analysis(num_attributes):
    # Tendência Central - média, mediana
    ct1 = pd.DataFrame(num_attributes.apply(np.mean)).T
    ct2 = pd.DataFrame(num_attributes.apply(np.median)).T

    # Dispersão - std, min, max, range, skew, kurtosis
    d1 = pd.DataFrame(num_attributes.apply(np.std)).T
    d2 = pd.DataFrame(num_attributes.apply(min)).T
    d3 = pd.DataFrame(num_attributes.apply(max)).T
    d4 = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min())).T
    d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T
    d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T

    # Concate
    m = pd.concat([d2, d3, d4, ct1, ct2, d1, d5, d6]).T.reset_index()
    m.columns = (['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis'])
    
    return m


def ml_error(model_name, y, yhat):
    mae = mean_absolute_error(y, yhat)
    mape = mean_absolute_percentage_error(y, yhat)
    rmse = np.sqrt(mean_squared_error(y, yhat))
    
    return pd.DataFrame({'Model Name': model_name,
                        'MAE': mae,
                        'MAPE': mape,
                        'RMSE': rmse}, index=[0])


def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.set_option('display.expand_frame_repr', False )
    pd.set_option('display.float_format', '{:.4f}'.format)
    
    sns.set()

In [3]:
jupyter_settings()

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


## 0.2 Loading data

In [4]:
df_raw = pd.read_csv('../data/raw/cars_train.csv', encoding='utf16', sep='\t')

In [5]:
df_raw.sample(3)

,id,num_fotos,marca,modelo,versao,ano_de_fabricacao,ano_modelo,hodometro,cambio,num_portas,tipo,blindado,cor,tipo_vendedor,cidade_vendedor,estado_vendedor,anunciante,entrega_delivery,troca,elegivel_revisao,dono_aceita_troca,veiculo_único_dono,revisoes_concessionaria,ipva_pago,veiculo_licenciado,garantia_de_fábrica,revisoes_dentro_agenda,veiculo_alienado,preco
8213,96041578899191612544991461815088024994,8.0000,FIAT,TORO,1.8 16V EVO FLEX FREEDOM AT6,2017,2017.0000,120383.0000,Automática,4,Picape,N,Branco,PF,Campinas,São Paulo (SP),Pessoa Física,False,False,False,Aceita troca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86987.3712
16229,228392305096878884886808044971278602629,8.0000,MITSUBISHI,OUTLANDER,2.2 4X4 16V DIESEL 4P AUTOMÁTICO,2016,2017.0000,90615.0000,Automática,4,Utilitário esportivo,N,Branco,PF,Mogi das Cruzes,São Paulo (SP),Pessoa Física,False,False,False,NaN,NaN,Todas as revisões feitas pela concessionária,IPVA pago,Licenciado,NaN,Todas as revisões feitas pela agenda do carro,NaN,232332.2300
18487,335000150022297571159769385522518716401,15.0000,HYUNDAI,HB20S,1.6 COMFORT PLUS 16V FLEX 4P AUTOMÁTICO,2015,2015.0000,85351.0000,Automática,4,Sedã,N,Prata,PJ,Presidente Prudente,São Paulo (SP),Loja,False,False,False,Aceita troca,NaN,NaN,IPVA pago,Licenciado,NaN,NaN,NaN,61256.4574


# 1.0 DESCRIÇÃO DOS DADOS

In [6]:
df1 = df_raw.copy()

## 1.1 Rename Columns

Reduzindo nome das colunas e retirando acentos para facilitar aplicações futuras.

In [7]:
df1 = df1.rename(columns = {'ano_de_fabricacao': 'ano_fabricacao',
                            'entrega_delivery': 'delivery',
                            'dono_aceita_troca': 'aceita_troca',
                            'veiculo_único_dono': 'unico_dono',
                            'ipva_pago': 'ipva',
                            'veiculo_licenciado': 'licenciado',
                            'garantia_de_fábrica': 'garantia_fabrica',
                            'revisoes_dentro_agenda': 'revisoes_agenda',
                            'veiculo_alienado': 'alienado'})

## 1.2 Data Dimensions

In [8]:
print(f'Number of Rows: {df1.shape[0]}')
print(f'Number of Cols: {df1.shape[1]}')

Number of Rows: 29584
Number of Cols: 29


## 1.3 Data Types

Análise inicial dos tipos de cada feature.

In [9]:
df1.dtypes

id                          object
num_fotos                  float64
marca                       object
modelo                      object
versao                      object
ano_fabricacao               int64
ano_modelo                 float64
hodometro                  float64
cambio                      object
num_portas                   int64
tipo                        object
blindado                    object
cor                         object
tipo_vendedor               object
cidade_vendedor             object
estado_vendedor             object
anunciante                  object
delivery                      bool
troca                         bool
elegivel_revisao              bool
aceita_troca                object
unico_dono                  object
revisoes_concessionaria     object
ipva                        object
licenciado                  object
garantia_fabrica            object
revisoes_agenda             object
alienado                   float64
preco               

## 1.4 Check NA

Conferindo valores nulos em cada feature.

In [10]:
df1.isna().sum()

id                             0
num_fotos                    177
marca                          0
modelo                         0
versao                         0
ano_fabricacao                 0
ano_modelo                     0
hodometro                      0
cambio                         0
num_portas                     0
tipo                           0
blindado                       0
cor                            0
tipo_vendedor                  0
cidade_vendedor                0
estado_vendedor                0
anunciante                     0
delivery                       0
troca                          0
elegivel_revisao               0
aceita_troca                7662
unico_dono                 19161
revisoes_concessionaria    20412
ipva                        9925
licenciado                 13678
garantia_fabrica           25219
revisoes_agenda            23674
alienado                   29584
preco                          0
dtype: int64

## 1.5 Descriptive Statistical

Ignorada no ciclo inicial.

# 2.0 FEATURE ENGINEERING

Ignorada no ciclo inicial.

# 3.0 FILTRAGEM DE VARIÁVEIS

Ignorada no ciclo inicial.

# 4.0 ANÁLISE EXPLORATÓRIA DOS DADOS

Ignorada no ciclo inicial.

# 5.0 DATA PREPARATION

In [122]:
df5 = df1.copy()

## 5.1 Separação de Treino e Teste

In [144]:
X = df5[df5.columns.drop(['id', 'preco'])]

# Transformacao logaritma na feature target
y = np.log1p(df5['preco'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=7)

## 5.2 Preparação dos dados de treino

In [145]:
# numero de fotos
X_train['num_fotos'].fillna(0, inplace=True)
mms_fotos = MinMaxScaler()
X_train['num_fotos'] = mms_fotos.fit_transform(X_train[['num_fotos']].values)

#marca, modelo, cor, cidade_vendedor, estado_vendedor
encoder = CountFrequencyEncoder(encoding_method='frequency',
                                variables=['marca', 'cor', 'modelo', 'cidade_vendedor', 'estado_vendedor'],
                                unseen='encode')
# fit the encoder
encoder.fit(X_train)

# transform
X_train = encoder.transform(X_train)

# ano_fabricacao
mms_fabricacao = MinMaxScaler()
X_train['ano_fabricacao'] = mms_fabricacao.fit_transform(X_train[['ano_fabricacao']].values)

# ano_modelo
mms_modelo = MinMaxScaler()
X_train['ano_modelo'] = mms_modelo.fit_transform(X_train[['ano_modelo']].values)

# hodometro
mms_hodometro = MinMaxScaler()
X_train['hodometro'] = mms_hodometro.fit_transform(X_train[['hodometro']].values)

# cambio
le_cambio = LabelEncoder()
X_train['cambio'] = le_cambio.fit_transform(X_train['cambio'])

# numero de portas
mms_portas = MinMaxScaler()
X_train['num_portas'] = mms_portas.fit_transform(X_train[['num_portas']].values)

# tipo
le_tipo = LabelEncoder()
X_train['tipo'] = le_tipo.fit_transform(X_train[['tipo']])

# blindado
le_blindado = LabelEncoder()
X_train['blindado'] = le_blindado.fit_transform(X_train[['blindado']].values)

# tipo_vendedor
le_vendedor = LabelEncoder()
X_train['tipo_vendedor'] = le_vendedor.fit_transform(X_train['tipo_vendedor'])

# anunciante
le_anunciante = LabelEncoder()
X_train['anunciante'] = le_anunciante.fit_transform(X_train['anunciante'])

# entrega_delivery
X_train.delivery = X_train.delivery.replace({True: 1, False:0})

# troca
X_train.troca = X_train.troca.replace({True: 1, False:0})

# elegivel revisao
X_train.elegivel_revisao = X_train.elegivel_revisao.replace({True: 1, False:0})

# aceita troca
X_train['aceita_troca'] = X_train['aceita_troca'].apply(lambda x: 0 if pd.isna(x) else 1)

# unico dono
X_train['unico_dono'] = X_train['unico_dono'].apply(lambda x: 0 if pd.isna(x) else 1)

# revisoes concessionaria
X_train['revisoes_concessionaria'] = X_train['aceita_troca'].apply(lambda x: 0 if pd.isna(x) else 1)

# ipva
X_train['ipva'] = X_train['ipva'].apply(lambda x: 0 if pd.isna(x) else 1)

# licenciado
X_train['licenciado'] = X_train['licenciado'].apply(lambda x: 0 if pd.isna(x) else 1)

# garantia_fabrica
X_train['garantia_fabrica'] = X_train['garantia_fabrica'].apply(lambda x: 0 if pd.isna(x) else 1)

# revisoes_agenda
X_train['revisoes_agenda'] = X_train['revisoes_agenda'].apply(lambda x: 0 if pd.isna(x) else 1)

/home/matheus/.pyenv/versions/3.10.7/envs/ge_ml/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/matheus/.pyenv/versions/3.10.7/envs/ge_ml/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## 5.4 Preparação dos dados de teste

In [146]:
# numero de fotos
X_test['num_fotos'].fillna(0, inplace=True)
X_test['num_fotos'] = mms_fotos.transform(X_test[['num_fotos']].values)

#marca, modelo, cor, cidade_vendedor, estado_vendedor
# transform
X_test = encoder.transform(X_test)

# ano_fabricacao
X_test['ano_fabricacao'] = mms_fabricacao.transform(X_test[['ano_fabricacao']].values)

# ano_modelo
X_test['ano_modelo'] = mms_modelo.transform(X_test[['ano_modelo']].values)

# hodometro
X_test['hodometro'] = mms_hodometro.transform(X_test[['hodometro']].values)

# cambio
X_test['cambio'] = le_cambio.transform(X_test['cambio'])

# numero de portas
X_test['num_portas'] = mms_portas.transform(X_test[['num_portas']].values)

# tipo
X_test['tipo'] = le_tipo.transform(X_test[['tipo']])

# blindado
X_test['blindado'] = le_blindado.transform(X_test[['blindado']].values)

# tipo_vendedor
X_test['tipo_vendedor'] = le_vendedor.transform(X_test['tipo_vendedor'])

# anunciante
X_test['anunciante'] = le_anunciante.transform(X_test['anunciante'])

# entrega_delivery
X_test.delivery = X_test.delivery.replace({True: 1, False:0})

# troca
X_test.troca = X_test.troca.replace({True: 1, False:0})

# elegivel revisao
X_test.elegivel_revisao = X_test.elegivel_revisao.replace({True: 1, False:0})

# aceita troca
X_test['aceita_troca'] = X_test['aceita_troca'].apply(lambda x: 0 if pd.isna(x) else 1)

# unico dono
X_test['unico_dono'] = X_test['unico_dono'].apply(lambda x: 0 if pd.isna(x) else 1)

# revisoes concessionaria
X_test['revisoes_concessionaria'] = X_test['aceita_troca'].apply(lambda x: 0 if pd.isna(x) else 1)

# ipva
X_test['ipva'] = X_test['ipva'].apply(lambda x: 0 if pd.isna(x) else 1)

# licenciado
X_test['licenciado'] = X_test['licenciado'].apply(lambda x: 0 if pd.isna(x) else 1)

# garantia_fabrica
X_test['garantia_fabrica'] = X_test['garantia_fabrica'].apply(lambda x: 0 if pd.isna(x) else 1)

# revisoes_agenda
X_test['revisoes_agenda'] = X_test['revisoes_agenda'].apply(lambda x: 0 if pd.isna(x) else 1)

/home/matheus/.pyenv/versions/3.10.7/envs/ge_ml/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/matheus/.pyenv/versions/3.10.7/envs/ge_ml/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# 6.0 FEATURE SELECTION

Para análise inicial, as features versao e alienado foram desconsideradas.

In [147]:
X_train = X_train[X_train.columns.drop(['versao', 'alienado'])]
X_test = X_test[X_test.columns.drop(['versao', 'alienado'])]

# 7.0 MACHINE LEARNING

## 7.1 Average Model

In [117]:
aux1 = df5[['marca', 'modelo']]
aux1['identificacao'] = aux1['marca'] + ' ' + aux1['modelo']
aux1['preco'] = y

X_mean_train, X_mean_test = train_test_split(aux1, test_size=0.25, random_state=7)

aux2 = X_mean_train[['identificacao',
             'preco']].groupby('identificacao').mean().reset_index().rename(columns={'preco': 'predictions'})

# merge
X_mean_test = pd.merge(X_mean_test, aux2, how='left', on='identificacao')

# Preenchendo valores faltantes para carros nao encontrados
X_mean_test['predictions'].fillna(aux2['predictions'].mean(), inplace=True)

# performance
baseline_result = ml_error('Average Model', np.expm1(X_mean_test['preco']), np.expm1(X_mean_test['predictions']))
baseline_result

/tmp/ipykernel_6867/2587287083.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aux1['identificacao'] = aux1['marca'] + ' ' + aux1['modelo']
/tmp/ipykernel_6867/2587287083.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aux1['preco'] = y


,Model Name,MAE,MAPE,RMSE
0,Average Model,37392.6966,0.3119,55391.4515


## 7.2 Linear Regression Model

In [148]:
# model
lr = LinearRegression().fit(X_train, y_train)

# prediction
yhat_lr = lr.predict(X_test)

# performace
lr_result = ml_error('Linear Regression', np.expm1(y_test), np.expm1(yhat_lr))
lr_result

,Model Name,MAE,MAPE,RMSE
0,Linear Regression,44563.8407,0.3698,65561.9199


## 7.3 Linear Regression Regularized Model - Lasso

In [154]:
# model
lrr = Lasso(alpha=0.005).fit(X_train, y_train)

# prediction
yhat_lrr = lrr.predict(X_test)

# performace
lrr_result = ml_error('Linear Regression - Lasso', np.expm1(y_test), np.expm1(yhat_lrr))
lrr_result

,Model Name,MAE,MAPE,RMSE
0,Linear Regression - Lasso,46984.5877,0.3833,70314.2234


## 7.4 Random Forest Regressor

In [174]:
# model
rf = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=7).fit(X_train, y_train)

# prediction
yhat_rf = rf.predict(X_test)

# performace
rf_result = ml_error('Random Forest Regression - Lasso', np.expm1(y_test), np.expm1(yhat_rf))
rf_result

,Model Name,MAE,MAPE,RMSE
0,Random Forest Regression - Lasso,28013.2247,0.2300,43431.5836


## 7.5 XGBoost Regressor

In [163]:
# model
model_xgb = xgb.XGBRegressor(n_estimators=100,
                             random_state=7).fit(X_train, y_train)

# prediction
yhat_xgb = model_xgb.predict(X_test)

# performance
xgb_result = ml_error('XGBoost Regressor', np.expm1(y_test), np.expm1(yhat_xgb))
xgb_result

,Model Name,MAE,MAPE,RMSE
0,XGBoost Regressor,27551.6606,0.2237,41999.3463


## 7.6 LightGBM Regressor

In [172]:
# model
model_lgbm = lgbm.LGBMRegressor(n_estimators=300).fit(X_train, y_train)

# prediction
yhat_lgbm = model_lgbm.predict(X_test)

# performance
lgbm_result = ml_error('LightGBM Regressor', np.expm1(y_test), np.expm1(yhat_lgbm))
lgbm_result

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003633 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 616
[LightGBM] [Info] Number of data points in the train set: 22188, number of used features: 23
[LightGBM] [Info] Start training from score 11.637541


,Model Name,MAE,MAPE,RMSE
0,LightGBM Regressor,27053.6408,0.2190,41845.9696


## 7.7 Compare Model's Performace

In [175]:
modelling_result_cv = pd.concat([baseline_result, lr_result, lrr_result, rf_result, xgb_result, lgbm_result])
modelling_result_cv.sort_values('RMSE')

,Model Name,MAE,MAPE,RMSE
0,LightGBM Regressor,27053.6408,0.2190,41845.9696
0,XGBoost Regressor,27551.6606,0.2237,41999.3463
0,Random Forest Regression - Lasso,28013.2247,0.2300,43431.5836
0,Average Model,37392.6966,0.3119,55391.4515
0,Linear Regression,44563.8407,0.3698,65561.9199
0,Linear Regression - Lasso,46984.5877,0.3833,70314.2234


# 8.0 HYPERPARAMETER FINE TUNING

# 9.0 TRADUÇÃO E INTERPRETAÇÃO DO ERRO

# 10.0 DEPLOY MODEL